<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/12_Decision_Analytics_Simplex_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Simplex Algorithm**

**BACKGROUND**

Here we implement the simplex algorithm to solve linear programs and assess if it generates the same results as the OR Tools solver.

In [ ]:
!pip install ortools

     |████████████████████████████████| 14.0MB 280kB/s 
     |████████████████████████████████| 133kB 54.7MB/s 
     |████████████████████████████████| 1.0MB 25.1MB/s 
ERROR: tensorflow-metadata 0.25.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
import numpy as np

**Task 1**

Implement a function to execute the pivoting operation on a matrix given the row and column of the pivot element.


In [ ]:
def execute_pivot(A,r,s):
    A[r,:] = A[r,:]/A[r,s]
    for i in range(A.shape[0]):
        if i!=r:
            A[i,:] = A[i,:] - A[i,s]*A[r,:]
    return

**Task 2**

Implement a function to execute the phase 2 of the simplex algorithm that optimises a tableau with explicit basis vectors.

In [ ]:
def simplex_phase_2(A):                    
    while True:    
        pivots = {}
        for j in range(1,A.shape[1]-1):
            cj = A[0,j]
            if cj>0:
                for i in range(1,A.shape[0]):
                    bi = A[i,-1]
                    if A[i,j]>0:
                        pivots[(i,j)] = bi/A[i,j]                    
        if len(pivots)==0:
            break        
        pivot = min(pivots.keys(), key=(lambda k: pivots[k]))
        execute_pivot(A,pivot[0],pivot[1])
    return


In [ ]:
def determine_basis(D):
    basis = []
    epsilon = 1e-6
    for j in range(1,D.shape[1]):
        if np.abs(D[0,j])<epsilon:
            zeros = sum(abs(D[1:D.shape[0],j])<epsilon)
            ones = sum(abs(D[1:D.shape[0],j]-1)<epsilon)
            if (ones==1) and (zeros==D.shape[0]-2):
                basis.append(j)
    return basis

**Task 3**

Implement a function to execute the phase 1 of the simplex algorithm that creates a tableau with explicit basis vectors.


In [ ]:
def simplex_phase_1(A):
    eb = np.sum(A[1:A.shape[0],-1])
    eA = np.sum(A[1:A.shape[0],1:A.shape[1]-1],axis=0)
        
    a = np.append(np.append(np.append([1], eA), np.zeros((1,A.shape[0]-1))), eb) 
    
    B = np.append(A[1:A.shape[0],0:A.shape[1]-1], np.eye(A.shape[0]-1), axis=1)
    C = np.append(B, A[1:A.shape[0],-1].reshape(B.shape[0],1), axis=1)
    D = np.append(a.reshape(1,len(a)),C,axis=0)    
    
    simplex_phase_2(D)
    
    epsilon = 1e-6
    if not abs(D[0,-1])<epsilon:
        print("No feasible solution!")
        return 
    
    while True:
        basis = determine_basis(D)        
        finished = True
        for j in basis:
            if j>=A.shape[1]-1:
                for i in range(1,D.shape[0]):
                    if abs(D[i,j]-1)<epsilon:
                        k = np.argmax(np.abs(D[i,1:A.shape[1]-1]))+1
                        execute_pivot(D,i,k)
                        finished = False
        if finished:
            break


    E = np.append(D[:,0:A.shape[1]-1], D[:,-1].reshape(D.shape[0],1), axis=1)

    for i in range(1,E.shape[1]-1):
        if i in basis:
            E[0,E.shape[1]-1] -= A[0,i]*D[np.argmax(D[:,i]),-1]
        else:
            E[0,i] = A[0,i]

    return E

**Task 4**

Create the input tableau for the diet problem. Run the tableau through the algorithm for phase 1 developed in task 3, and then through the phase 2 developed in task 2.

In [ ]:
# Diet Problem

In [ ]:
A = np.array([[1,-9,-7,0,0,0,0],
                  [0,2,4,-1,0,0,12],
                  [0,5,3,0,-1,0,15],
                  [0,4,1,0,0,-1,8]], dtype=np.float32)


In [ ]:
 print(A)

[[ 1. -9. -7.  0.  0.  0.  0.]
 [ 0.  2.  4. -1.  0.  0. 12.]
 [ 0.  5.  3.  0. -1.  0. 15.]
 [ 0.  4.  1.  0.  0. -1.  8.]]


In [ ]:
A = simplex_phase_1(A)

In [ ]:
simplex_phase_2(A)

In [ ]:
print(np.array(100*A, dtype=np.int32)/100.0)

[[ 1.    0.    0.    0.    0.    0.   30.42]
 [ 0.    0.    1.   -0.35  0.14  0.    2.14]
 [ 0.    0.    0.    0.5  -1.    1.    1.  ]
 [ 0.    1.    0.    0.21 -0.28  0.    1.71]]
